In [13]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

In [10]:
df = pd.read_csv(r"D:\TEL-U\BANGKIT!\NARASPEAK\DATASET\C4_200M.tsv-00000-of-00010", delimiter='\t', nrows=5000)
df.columns = ["Incorrect", "Correct"]

In [11]:
df

,Incorrect,Correct
0,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,"Fairy Or Not, I'm the Godmother: no just look,...","Fairy Or Not, I'm the Godmother: Not just a lo..."
...,...,...
4995,"Klaviyo, Yotpo, Yext, Dashbot and more live.","Klaviyo, Yotpo, Yext, Dashbot, and more."
4996,"Lead And Zinc Flotation Process, Gold, silver,...","Lead And Zinc Flotation Process, Gold, silver,..."
4997,So far as I remember I read something similar;...,As far as I remember I read something similar ...
4998,Every US Presient who repeat Tavistock fabrica...,Each U.S. President who repeated the Tavistock...


In [14]:
def remove_spaces(text) : 
    text = re.sub(r" '(\w)",r"'\1",text)
    text = re.sub(r" \,",",",text)
    text = re.sub(r" \.+",".",text)
    text = re.sub(r" \!+","!",text)
    text = re.sub(r" \?+","?",text)
    text = re.sub(" n't","n't",text)
    text = re.sub("[\(\)\;\_\^\`\/]","",text)
    
    return text

In [15]:
def decontract(text) : 
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    
    return text

In [16]:
def WrongSentence_preprocessing(col) : 
    df[col] = df[col].astype(str) 
    df[col] = df[col].apply(lambda x:x.lower()) 
    df[col] = df[col].apply(lambda x : re.sub("\n","",x)) 
    df[col] = df[col].apply(lambda x : remove_spaces(x)) 
    df[col] = df[col].apply(lambda x : re.sub(r"\.+",".",x)) 
    df[col] = df[col].apply(lambda x : re.sub(r"\!+","!",x)) 
    df[col] = df[col].apply(lambda x : decontract(x)) 
    df[col] = df[col].apply(lambda x: re.sub("[^A-Za-z\s]","",x)) 
    df[col] = df[col].apply(lambda x: x.replace("\s+"," "))
    df[col] = df[col].apply(lambda x: " ".join([word for word in x.split()]))
    return df

def TrueSentence_preprocessing(col) : 
    df[col] = df[col].astype(str) 
    df[col] = df[col].apply(lambda x:x.lower()) 
    df[col] = df[col].apply(lambda x : re.sub("\n","",x)) 
    df[col] = df[col].apply(lambda x : remove_spaces(x)) 
    df[col] = df[col].apply(lambda x : re.sub(r"\.+",".",x)) 
    df[col] = df[col].apply(lambda x : re.sub(r"\!+","!",x)) 
    df[col] = df[col].apply(lambda x : decontract(x)) 
    df[col] = df[col].apply(lambda x: re.sub("[^A-Za-z\s]","",x)) 
    df[col] = df[col].apply(lambda x: re.sub(r'\d','',x))
    df[col] = df[col].apply(lambda x: re.sub(r'\s+',' ',x))
    df[col] = df[col].apply(lambda x: re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,।]", "", x))
    df[col] = df[col].apply(lambda x: x.strip())
    df[col] = "<sos> " + df[col] + " <eos>"
    return df

In [17]:
df = WrongSentence_preprocessing('Incorrect') 
df = TrueSentence_preprocessing('Correct')


In [22]:

df['Incorrect_char_count'] = df['Incorrect'].astype('str').apply(lambda x:len(x))
df['Correct_char_count'] = df['Correct'].astype('str').apply(lambda x:len(x))

In [24]:
df['Incorrect_char_count']  = df['Incorrect'].astype('str').apply(lambda x:len(x.split()))
df['Correct_char_count']  = df['Correct'].astype('str').apply(lambda x:len(x.split()))

In [26]:
df

,Incorrect,Correct,correct_char_count,incorrect_char_count,Incorrect_char_count,Correct_char_count
0,the effect of widespread dud targets two face ...,<sos> the effect of widespread dud targets two...,86,98,15,17
1,tax on sales of stores for non residents are s...,<sos> capital gains tax on the sale of propert...,309,342,55,61
2,much many brands and sellers still in the market,<sos> many brands and sellers still in the mar...,48,55,9,10
3,this is is the latest maintenance release of s...,<sos> this is is the latest maintenance releas...,50,62,9,11
4,fairy or not i am the godmother no just look b...,<sos> fairy or not i am the godmother not just...,91,109,19,23
...,...,...,...,...,...,...
4995,klaviyo yotpo yext dashbot and more live,<sos> klaviyo yotpo yext dashbot and more <eos>,40,47,7,8
4996,lead and zinc flotation process gold silver co...,<sos> lead and zinc flotation process gold sil...,282,297,45,47
4997,so far as i remember i read something similar ...,<sos> as far as i remember i read something si...,100,126,19,23
4998,every us presient who repeat tavistock fabrica...,<sos> each us president who repeated the tavis...,122,146,19,23


In [28]:

pd.DataFrame(df.isna().sum(),columns=['missing_count'])

,missing_count
Incorrect,0
Correct,0
correct_char_count,0
incorrect_char_count,0
Incorrect_char_count,0
Correct_char_count,0


In [36]:
df.shape

(5000, 6)

In [38]:
print(f'total number of duplicates: {df.duplicated().sum()}')
     

total number of duplicates: 0


In [39]:

df[df['incorrect_char_count']<2].shape
     

(0, 6)

In [40]:
df = df[df['incorrect_char_count']>2].reset_index(drop=True)

In [41]:
df[df['correct_char_count']<2].shape

(0, 6)

In [47]:

df[df['correct_char_count']<20].sample(10)

,Incorrect,Correct,correct_char_count,incorrect_char_count,Incorrect_char_count,Correct_char_count
2572,weight lbs oz,<sos> weight lbs oz <eos>,13,25,3,5
4659,trinity p the joker,<sos> trinity p the joker <eos>,19,31,4,6
2603,image it x in,<sos> image x in <eos>,13,22,4,5
806,so i have some here,<sos> so i have some here <eos>,19,31,5,7
381,bbb would weigh,<sos> bbb would weigh pounds <eos>,15,34,3,6
937,you are my child,<sos> user you are my child <eos>,16,33,4,7
1652,you rate mm mm jeep,<sos> pictured at mm and mm <eos>,19,33,5,7
3355,sq ft,<sos> sq ft <eos>,5,17,2,4
316,p ill,<sos> description p ill <eos>,5,29,2,5
4511,will i not mete,<sos> did i not mete <eos>,15,26,4,6


In [48]:
df

,Incorrect,Correct,correct_char_count,incorrect_char_count,Incorrect_char_count,Correct_char_count
0,the effect of widespread dud targets two face ...,<sos> the effect of widespread dud targets two...,86,98,15,17
1,tax on sales of stores for non residents are s...,<sos> capital gains tax on the sale of propert...,309,342,55,61
2,much many brands and sellers still in the market,<sos> many brands and sellers still in the mar...,48,55,9,10
3,this is is the latest maintenance release of s...,<sos> this is is the latest maintenance releas...,50,62,9,11
4,fairy or not i am the godmother no just look b...,<sos> fairy or not i am the godmother not just...,91,109,19,23
...,...,...,...,...,...,...
4995,klaviyo yotpo yext dashbot and more live,<sos> klaviyo yotpo yext dashbot and more <eos>,40,47,7,8
4996,lead and zinc flotation process gold silver co...,<sos> lead and zinc flotation process gold sil...,282,297,45,47
4997,so far as i remember i read something similar ...,<sos> as far as i remember i read something si...,100,126,19,23
4998,every us presient who repeat tavistock fabrica...,<sos> each us president who repeated the tavis...,122,146,19,23
